In [1]:
from aoc_utils import *

import pandas as pd
import numpy as np

from aocd import get_data, submit

In [2]:
year = 2023
day = 21

In [34]:
dat = get_data(day=day, year=year, block=True)

In [39]:
dat = '''...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........'''

In [115]:
g = Grid(dat)

max_c, max_r = max(g)
len_c = max_c + 1
len_r  = max_r + 1

for k,v in g.items():
    if v == 'S':
        start = k
        g[k] = '.'
        break
#visited = set()

from functools import cache


visited = visit_count(g, start)

In [134]:
visited

array([[10.,  9.,  8.,  7.,  6.,  7.,  8.,  7.,  8.,  9., 10.],
       [11., 10., nan,  6.,  5., nan, nan,  6., nan, nan,  9.],
       [12., 11., nan, nan,  4., nan, nan,  5., nan, nan,  8.],
       [13., 12., nan,  4.,  3.,  2.,  3.,  4.,  5.,  6.,  7.],
       [12., 13., 14., nan, nan,  1.,  2.,  3., nan,  7.,  8.],
       [11., nan, nan,  2.,  1.,  0., nan,  4.,  5., nan,  9.],
       [10., nan, nan,  3., nan, nan,  6.,  5., nan, nan, 10.],
       [ 9., nan,  5.,  4.,  5., nan,  7., nan, nan, 12., 11.],
       [ 8.,  7.,  6., nan,  6., nan,  8., nan, nan, nan, 12.],
       [ 9., nan, nan,  8.,  7., nan, nan, 12., nan, nan, 13.],
       [10., 11., 10.,  9.,  8.,  9., 10., 11., 12., 13., 14.]])

In [283]:
def visit_count(start):
    max_c, max_r = max(g)
    len_c = max_c + 1
    len_r  = max_r + 1
    visited = np.zeros((len_c, len_r))*np.nan
    #visited = {}
    to_search = {start}
    
    i = 0
    while to_search:
        new_search = set()
        for p in to_search:
            if np.isnan(visited[*p]):
                visited[*p] = i #str(i).rjust(3)
            for n in g.neighbors(p):
                if g[n] == '.' and np.isnan(visited[*n]):
                    new_search.add(n)
        to_search = new_search
        i += 1
    return visited
    
def extrapolate(v, dir): 
    # dir can be ({-1,0,1}, {-1,0,1})
    if dir[0] == 0:
        if dir[1] == 1:
            r, rw = max_r, 0
        else:
            r, rw = 0, max_r
        c = range(len_c)
        rc = zip(c, [r]*len_c)
        rcw = zip(c, [rw]*len_c)
        rc_iter = zip(rc, rcw)
    elif dir[1] == 0:
        if dir[0] == 1:
            c, cw =max_c, 0
        else:
            c, cw = 0, max_c
        r = range(len_r)
        rc = zip([c]*len_r, r)
        rcw = zip([cw]*len_r, r)
        rc_iter = zip(rc, rcw)
    else:
        v1 = extrapolate(v, (0, dir[1]))
        v1 = extrapolate(v1, (dir[0], 0))
        v2 = extrapolate(v, (dir[0], 0))
        v2 = extrapolate(v2, (0, dir[1]))
        return np.minimum(v1, v2)
        
    vs = []
    for rc, rcw in rc_iter: #range(len_r): 
        #print(rc, rcw)
        #print(1 + v[*rc] + visit_count(rcw))
        vs.append( 1 + v[*rc] + visit_count(rcw))
    min_v = reduce(np.minimum, vs)
    return min_v

dirs = {(0,1), (0,-1), (1,0), (-1,0), (1,1), (-1,1), (1,-1), (-1,-1)}

def border_elems(v, dir=(0,0)):
    res = []
    if dir[1] == 1 or dir == (0,0):
        res.append(v[:,-1])
    if dir[1] == -1 or dir == (0,0):
        res.append(v[:,0])
    if dir[0] == 1 or dir == (0,0):
        res.append(v[-1,:])
    if dir[0] == -1 or dir == (0,0):
        res.append(v[0,:])
    return np.concatenate(res, axis=None)

def sub_elems(d, b):
    for k,v in d.items():
        d[k] = v - b
    return d

def expand(d):
    dim = max(max(d))
    d_new = {}
    b_min = np.inf
    b_max = 0
    for p, v in d.items():
        for dir in dirs:
            if max(map(abs, add2(p, dir))) == dim + 1:
                # extrapolate 
                v_ext = extrapolate(v, dir)
                border = border_elems(v_ext, dir)
                b_min = min(b_min, min(border))
                b_max = max(b_max, max(border))
                
                d_new[add2(p,dir)] = v_ext
    return d_new, b_min, b_max

def count(d, nmax=np.inf):
    c = [0,0]
    for v in d.values():
        c[0] += np.sum((v % 2 == 0) & (v <= nmax))
        c[1] += np.sum((v % 2 == 1) & (v <= nmax))
    return c

def add_count(c, d, offset, nmax=np.inf):
    c_new = count(d, nmax)
    if offset:
        c_new = list(reversed(c_new))
    return add2(c, c_new)

In [358]:
n = 51
c = [0,0]
v = visit_count(start)
b_min = min(border_elems(v))
b_max = max(border_elems(v))
n_min = b_min
n_max = b_max
n_min_prev = 0
d = {(0,0):v}
offset = False

# while n_max < n:
#     n_min_prev = n_min
#     c = add_count(c, d, offset)
#     d = sub_elems(d, b_min)
#     offset = bool(b_min % 2)
#     d, b_min, b_max = expand(d)
#     n_min += b_min
#     n_max += b_max
    
while n_min < n:
    n_min -= b_min
    n_max -= b_max
    d_new, b_min, b_max = expand(d)
    d.update(d_new)
    n_min += b_min
    n_max += b_max
        
    
nrel = n - n_min_prev
c = add_count(c, d, offset, nrel)

c[n % 2]
c

(1407, 1444)

In [359]:
n_min

55.0

In [360]:
Counter([max([abs(i) for i in k]) for k in d.keys()])

Counter({4: 32, 3: 24, 2: 16, 1: 8, 0: 1})

In [361]:
dim = max(max(d))
d_a = {(k[0]+dim,k[1]+dim):v for k,v in d.items()}
vis = np.zeros([dim*2 + 1, dim*2 + 1])
vis = [[None for i in range(dim*2+1)] for j in range(dim*2+1)]
for (i,j),v in d_a.items():
    vis[i][j] = v
vis = np.block(vis)
pd.DataFrame(vis).replace(np.nan, -1).astype(int).to_csv('vis.csv')

In [364]:
min(border_elems(vis))

56.0

In [235]:
b_min

7.0

In [234]:
d[(-1,1)]

array([[22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32.],
       [21., 22., nan, 26., 27., nan, nan, 28., nan, nan, 31.],
       [20., 21., nan, nan, 26., nan, nan, 27., nan, nan, 30.],
       [19., 20., nan, 26., 25., 24., 25., 26., 27., 28., 29.],
       [18., 19., 20., nan, nan, 23., 24., 25., nan, 29., 28.],
       [17., nan, nan, 20., 21., 22., nan, 26., 27., nan, 27.],
       [16., nan, nan, 19., nan, nan, 28., 27., nan, nan, 26.],
       [15., nan, 17., 18., 19., nan, 29., nan, nan, 26., 25.],
       [14., 15., 16., nan, 18., nan, 30., nan, nan, nan, 24.],
       [13., nan, nan, 16., 17., nan, nan, 20., nan, nan, 23.],
       [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.]])

In [232]:
extrapolate(v, (-1,1))

array([[22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32.],
       [21., 22., nan, 26., 27., nan, nan, 28., nan, nan, 31.],
       [20., 21., nan, nan, 26., nan, nan, 27., nan, nan, 30.],
       [19., 20., nan, 26., 25., 24., 25., 26., 27., 28., 29.],
       [18., 19., 20., nan, nan, 23., 24., 25., nan, 29., 28.],
       [17., nan, nan, 20., 21., 22., nan, 26., 27., nan, 27.],
       [16., nan, nan, 19., nan, nan, 28., 27., nan, nan, 26.],
       [15., nan, 17., 18., 19., nan, 29., nan, nan, 26., 25.],
       [14., 15., 16., nan, 18., nan, 30., nan, nan, nan, 24.],
       [13., nan, nan, 16., 17., nan, nan, 20., nan, nan, 23.],
       [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.]])

In [178]:
len(d)

24

In [188]:
c

(1986, 1907)

In [171]:
np.minimum(min_v, -1*min_v)

array([[-10.,  -9.,  -8.,  -7.,  -8.,  -9., -10., -11., -12., -13., -14.],
       [-11., -10.,  nan,  -8.,  -9.,  nan,  nan, -12.,  nan,  nan, -15.],
       [-12., -11.,  nan,  nan, -10.,  nan,  nan, -13.,  nan,  nan, -16.],
       [-13., -12.,  nan, -12., -11., -12., -13., -14., -15., -16., -17.],
       [-14., -13., -14.,  nan,  nan, -13., -14., -15.,  nan, -17., -18.],
       [-15.,  nan,  nan, -16., -15., -14.,  nan, -16., -17.,  nan, -19.],
       [-16.,  nan,  nan, -17.,  nan,  nan, -18., -17.,  nan,  nan, -20.],
       [-17.,  nan, -19., -18., -19.,  nan, -19.,  nan,  nan, -22., -21.],
       [-18., -19., -20.,  nan, -20.,  nan, -20.,  nan,  nan,  nan, -22.],
       [-19.,  nan,  nan, -22., -21.,  nan,  nan, -26.,  nan,  nan, -23.],
       [-20., -21., -22., -23., -22., -23., -24., -25., -26., -25., -24.]])

In [164]:
from itertools import zip_longest
list(zip_longest(range(5), [4]))

[(0, 4), (1, None), (2, None), (3, None), (4, None)]

In [153]:
np.sum((min_v % 2 == 0) & (min_v < 20))

28

In [145]:

border_elems(min_v)

array([10.,  9.,  8.,  7.,  8.,  9., 10., 11., 12., 13., 14., 11., 15.,
       12., 16., 13., 17., 14., 18., 15., 19., 16., 20., 17., 21., 18.,
       22., 19., 23., 20., 21., 22., 23., 22., 23., 24., 25., 26., 25.,
       24.])

In [139]:
np.sum(min_v % 2 == 0)

42

In [112]:
Grid(min_v, default=' #').print(sep=' ')

10 11 12 13 14 15 16 17 18 19 20
11 12 13 14 15  #  #  # 19  # 21
10  #  #  # 16  #  # 19 20  # 22
9 10  # 12  # 16 17 18  # 22 23
8 9 10 11  # 15  # 19 20 21 22
9  #  # 12 13 14  #  #  #  # 23
10  #  # 13 14  # 18 19 20  # 24
11 12 13 14 15 16 17  #  # 26 25
12  #  # 15  # 17  #  #  #  # 26
13  #  # 16 17  #  # 22  #  # 25
14 15 16 17 18 19 20 21 22 23 24


In [37]:
p1 = res

In [38]:
submit(p1, day=day, part='a', year=year)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [83]:
g = Grid(dat)

max_c, max_r = max(g)
max_c += 1
max_r += 1

for k,v in g.items():
    if v == 'S':
        start = k
        g[k] = '.'
        break

def wrap_add(p: Point, q: Point, max_c, max_r) -> Point: 
    x = p[0] + q[0]
    y = p[1] + q[1]
    return (x % max_c, y % max_r), (x // max_c, y // max_r)
    
from functools import cache

def inf_neighbors(p,max_c,max_r):
    return [wrap_add(p, Δ,max_c,max_r) for Δ in directions4]

@cache
def valid_neighbors(p, p_wrap, max_c, max_r):
    new_search = set()
    for n_coords,n_wrap in inf_neighbors(p,max_c,max_r):
        if g[n_coords] == '.':
            n_wrap = add2(p_wrap,n_wrap)
            new_search.add((n_coords, n_wrap))
    return new_search

In [84]:
n = 500
to_search = [(start,(0,0))]
visited = set()
odd_count = 0
even_count = 0
for i in range(n):
    new_search = set()
    visited.add
    for p,p_wrap in to_search:
        new_search.update(valid_neighbors(p, p_wrap, max_c, max_r))
    res = len(new_search)
    to_search = new_search
res

KeyboardInterrupt: 

In [4]:
set.pop?

Docstring:
Remove and return an arbitrary set element.
Raises KeyError if the set is empty.
Type:      method_descriptor

In [8]:
submit(p2, day=day, part='b', year=year)

Part b already solved with same answer: 55358
